In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from src.const import DATA_HEAD

data_filename = DATA_HEAD / "STORM_preprocessed_medianfill_1.csv"
base_df = pd.read_csv(str(data_filename), index_col=0)
base_df.head()

,ID,EventName,Year,Month,DisasterType,DisasterSubtype,MainLandfallLocation,Flood,Slide,OFDAResponse,Appeal,Declaration,LandfallMagnitude(kph),LandfallPressure(mb),TotalDeaths,NoInjured,TotalDamage(000US$),TotalDamageAdjusted(000US$),CPI
0,STORM_NAN_1953,NaN,1953,9,Storm,Tropical cyclone,8,0,0,0,0,0,92,989,1000,20,19400,211880,9.156133
1,STORM_VIOLET_1964,Violet,1964,9,Storm,Tropical cyclone,4,1,1,0,0,0,92,989,18,20,10000,94354,10.598376
2,STORM_IRIS_1964,Iris,1964,11,Storm,Tropical cyclone,5,1,0,0,0,0,150,960,5100,20,70000,660479,10.598376
3,STORM_JOAN_1964,Joan,1964,11,Storm,Tropical cyclone,5,1,0,0,0,0,130,980,2500,20,15000,141531,10.598376
4,STORM_KATE_1964,Kate,1964,11,Storm,Tropical cyclone,6,1,0,0,0,0,130,970,0,20,3000,28306,10.598376



# I. Plan

1. **Featue importance**
2. **Categorical classification**
3. **damage prediction**

# 1. Feature importance

# 2. Categorical classification


| **Mô hình**               | **Ưu điểm**                            | **Nhược điểm**                          | **Khi nào dùng?**                       |
|---------------------------|----------------------------------------|----------------------------------------|----------------------------------------|
| Logistic Regression       | Đơn giản, dễ hiểu                      | Chỉ hiệu quả với dữ liệu tuyến tính    | Dữ liệu tuyến tính                     |
| KNN                       | Dễ triển khai                          | Dự đoán chậm với dữ liệu lớn           | Dữ liệu nhỏ, không tuyến tính          |
| Decision Tree             | Dễ trực quan hóa                       | Dễ bị overfitting                      | Phân loại nhanh                        |
| Random Forest             | Giảm overfitting                       | Tốn tài nguyên hơn Decision Tree       | Dữ liệu phức tạp                       |
| SVM                       | Hiệu quả với không gian chiều cao      | Khó mở rộng cho dữ liệu lớn            | Dữ liệu chiều cao                      |
| Naive Bayes               | Nhanh, hiệu quả                        | Giả định độc lập không luôn đúng       | Phân loại văn bản                      |
| Gradient Boosting         | Kết quả tốt                            | Tốn tài nguyên                         | Dữ liệu phức tạp, cần hiệu quả cao     |
| Neural Network (MLP)      | Học được mẫu phức tạp                  | Cần nhiều tài nguyên                   | Khi có nhiều dữ liệu                   |

**Model selection**
1. `Logistic Regression`
2. `KNN`
3. `Random Forest`
   - Random Forest works well with data that has both linear and non-linear features. 
   - It does not require normalization or transformation of the data (i.e., the data does not need to follow the same distribution). 
   - The model minimizes overfitting by aggregating from multiple decision trees.
4. `SVM with rbf or Polynomial kernel`
   - SVM with RBF or Polynomial kernel can turn non-linear data into linear in higher space. 
   - Suitable if data is moderate in size (SVM does not scale well for large data). 

## 2.1 SLIDE

In [3]:
from sklearn.model_selection import train_test_split

from src.const import CATEGORICAL_TARGETS, ATTRIBUTES

evaluate_dict = dict()

print(f"CATEGORICAL_TARGETS :{CATEGORICAL_TARGETS}")
print(f"ATTRIBUTES :{ATTRIBUTES}")

slide_df = base_df.copy()
X = slide_df[ATTRIBUTES]
y = slide_df[CATEGORICAL_TARGETS[0]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(f"Train size {len(X_train)}")
print(f"Test size: {len(X_test)}")

CATEGORICAL_TARGETS :['Flood', 'Slide']
ATTRIBUTES :['Year', 'Month', 'MainLandfallLocation', 'OFDAResponse', 'Appeal', 'Declaration', 'LandfallMagnitude(kph)', 'LandfallPressure(mb)']
Train size 165
Test size: 19


### 2.1.1 Logistic regression

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from src.utils import evaluate


epochs = 1000
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
slide_lr = LogisticRegression(max_iter=epochs)
slide_lr.fit(X_train, y_train)

eval_values = evaluate(slide_lr, X_test, y_test, threshold=0.2) # thresh = 0.3 to bias recall
evaluate_dict[slide_lr.__class__.__name__] = eval_values

evaluate_dict

{'LogisticRegression': {'confusion_matrix': array([[ 0,  5],
         [ 0, 14]]),
  'accuracy': 0.74,
  'precision': np.float64(0.74),
  'recall': np.float64(1.0)}}

# 3. Damage prediction